# Initializing

In [29]:
from IPython.display import HTML

HTML("""
<button id="code-show-switch-btn">スクリプトを非表示にする</button>

<script>
var code_show = true;

function switch_display_setting() {
    var switch_btn = $("#code-show-switch-btn");
    if (code_show) {
        $("div.input").hide();
        code_show = false;
        switch_btn.text("スクリプトを表示する");
    }else {
        $("div.input").show();
        code_show = true;
        switch_btn.text("スクリプトを非表示にする");
    }
}

$("#code-show-switch-btn").click(switch_display_setting);
</script>
""")

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

if sys.platform == "darwin":
    # For mac-OS environment (Kobayahsi)
    import stata_setup
    # stata_setup.config("/Applications/stata", "mp")
    stata_setup.config("/Applications/stata 17", "mp")
elif os.name == "nt":
    # For windows environment (kakehi)
    import markdown
    import warnings
    warnings.filterwarnings('ignore')
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.max_columns', 100)
    pd.options.display.precision = 4
    import stata_setup
    stata_setup.config("C://Program Files//Stata18", "mp")
else:
    raise ValueError("Unsupported platform")


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      18.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2023 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 2-core  perpetual
Serial number: 501806329854
  Licensed to: Haruo Kakehi
               Keio Univertisy

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


## Setup

In [31]:
%%stata
qui{
use ../data/suicide_analysis_data_static,clear
xtset citycode covid
    
/* Local macro */
local absvars="citycode covid"
local clsvars="citycode"
local absvars_ind "i.citycode i.covid"    
local gmm2="gmm2s"
local log_="log_"
sum suicide
local suicide_t_m=r(mean)
sum suicide_m
local suicide_m_m=r(mean)
sum suicide_f
local suicide_f_m=r(mean)
sum `log_'active_opening_ratio
local log_active_opening_ratio_m=r(mean)
sum `log_'unemployment_rate
local log_unemployment_rate_m=r(mean)
sum `log_'patient
local log_patient_m=r(mean)
sum log_active_opening_weighted
local log_active_opening_weighted_m=r(mean)
sum log_bankruptcy_debt
local log_bankruptcy_debt_m=r(mean)

* bundle commute time IV variable
local commute_time_ratio_ivs = "commuter_ratio_no commuter_ratio_u15 commuter_ratio_15_30 commuter_ratio_30_45 commuter_ratio_45_60 commuter_ratio_60_90 commuter_ratio_90_120 commuter_ratio_o120 "

global industry_large_share_2015 "industry_large_1_15 industry_large_2_15 industry_large_3_15" 
global other_level_controls "young_rate middle_rate elderly_rate employment_rate employee_rate self_employee_rate labor_force_rate single_household_ratio"
global other_level_effective_controls " industry_large_3_15 young_rate self_employee_rate labor_force_rate single_household_ratio"


* For Tables
local txt_u20="Under 20"
local txt_20_29="20-29"
local txt_30_39="30-39"
local txt_40_49="40-49"
local txt_50_59="50-59"
local txt_o60="60 or over"

/* Transformation of log(1+var) */
local paracomp_ct_1st="(D_commuting_time_iv: _b[D1.commuting_time_iv])  "

local paracomp_f_stayhome="(stay_home: _b[D_stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) "
local paracomp_f_cr_reduced="(commuter_ratio: _b[commuter_ratio]*(1+`suicide_f_m')/(`suicide_f_m'))  "
local paracomp_f_ct_reduced="(D_commuting_time_iv: _b[D_commuting_time_iv]*(1+`suicide_f_m')/(`suicide_f_m'))  "

local paracomp_m_stayhome="(stay_home: _b[D_stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) "
local paracomp_m_cr_reduced="(commuter_ratio: _b[commuter_ratio]*(1+`suicide_m_m')/(`suicide_m_m'))  "
local paracomp_m_ct_reduced="(D_commuting_time_iv: _b[D_commuting_time_iv]*(1+`suicide_m_m')/(`suicide_m_m'))  "
}


. qui{

. 


## Note
- The Rotemberg decomposition is conducted in {ファイル名}.
- The pre-trend analysis is conducted in {ファイル名}.

## Baseline: Commuting-Time-Telework-ShiftShare

In [32]:
%%stata
qui{
foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv), gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv), gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd , 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);
};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      4.823
        (2.026)
        [0.017]
---------------
Eff~c   1964.183
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      1.447
        (0.747)
        [0.053]
---------------
Eff~c   1964.183
$\t~%     23.109
Obs~n       1241
---------------

. 


## Other Control

### city

In [33]:
%%stata
qui{

foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv) is_city, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_city

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_city

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_city

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_city

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_city
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_city

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_city

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv) is_city, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_city

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_city

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_city

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_city

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_city
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_city

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_city
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_city, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_city, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);

};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      4.532
        (2.012)
        [0.024]
---------------
Eff~c   1990.049
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      1.422
        (0.752)
        [0.059]
---------------
Eff~c   1990.049
$\t~%     23.109
Obs~n       1241
---------------

. 


### Industry controls

In [34]:
%%stata
qui{

foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv) $industry_large_share_2015, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_ind

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_ind

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_ind

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_ind

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_ind
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_ind

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_ind
/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv) $industry_large_share_2015, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_ind

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_ind

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_ind

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_ind

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_ind
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_ind

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_ind

}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_ind, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_ind, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") varwidth(4) modelwidth(10);


};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      4.192
        (2.750)
        [0.127]
---------------
Eff~c   1164.216
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      1.603
        (1.079)
        [0.137]
---------------
Eff~c   1164.216
$\t~%     23.109
Obs~n       1241
---------------

. 


### city + industry control + other level control

In [35]:
%%stata
qui{

foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv) $industry_large_share_2015 $other_level_effective_controls is_city, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_cio

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_cio

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_cio

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_cio

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_cio
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_cio

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_cio

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv) $industry_large_share_2015 $other_level_effective_controls is_city, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_cio

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_cio

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_cio

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_cio

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_cio
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_cio

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_cio
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_cio, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_cio, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);



};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      6.333
        (3.189)
        [0.047]
---------------
Eff~c    836.720
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      0.841
        (1.139)
        [0.460]
---------------
Eff~c    836.720
$\t~%     23.109
Obs~n       1241
---------------

. 


### Δunemployment

In [36]:
%%stata
qui{

foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv) D.log_unemployment_rate, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_uemp

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_uemp

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_uemp

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_uemp

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_uemp
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_uemp

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_uemp

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv) D.log_unemployment_rate, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_uemp

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_uemp

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_uemp

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_uemp

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_uemp
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_uemp

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_uemp
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_uemp, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_uemp, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);


};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      4.760
        (2.126)
        [0.025]
---------------
Eff~c   1677.856
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      0.968
        (0.852)
        [0.256]
---------------
Eff~c   1677.856
$\t~%     23.109
Obs~n       1241
---------------

. 


### ΔActive job opening ratio

In [37]:
%%stata
qui{

foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv) D.log_active_opening_ratio, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_ajor

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_ajor

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_ajor

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_ajor

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_ajor
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_ajor

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_ajor

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv) D.log_active_opening_ratio, gmm2s first cluster(`clsvars')  savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_ajor

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_ajor

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_ajor

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_ajor

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_ajor
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_ajor

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_ajor
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_ajor, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

#delimit ;
noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_ajor, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);

};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      5.163
        (2.224)
        [0.020]
---------------
Eff~c   1654.951
$\t~%     23.109
Obs~n       1239
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      1.572
        (0.818)
        [0.054]
---------------
Eff~c   1654.951
$\t~%     23.109
Obs~n       1239
---------------

. 


### ΔUnemployment ＋ ΔActive job opening ratio

In [38]:
%%stata
qui{

foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv) D.log_unemployment_rate D.log_active_opening_ratio, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_ua

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_ua

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_ua

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_ua

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_ua
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_ua

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_ua

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv) D.log_unemployment_rate D.log_active_opening_ratio, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_ua

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_ua

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_ua

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_ua

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_ua
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_ua

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_ua
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_ua, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_ua, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);



};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      5.029
        (2.302)
        [0.029]
---------------
Eff~c   1535.249
$\t~%     23.109
Obs~n       1239
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      1.143
        (0.893)
        [0.201]
---------------
Eff~c   1535.249
$\t~%     23.109
Obs~n       1239
---------------

. 


### Δcovid cases ＋ Δcovid death cases

In [39]:
%%stata
qui{

foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D.commuting_time_iv) D.log_patient D.log_covid_death_cases, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_cd

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_cd

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_cd

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_cd

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_cd
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_cd

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_cd

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D.commuting_time_iv) D.log_patient D.log_covid_death_cases, gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_cd

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_cd

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_cd

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_cd

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_cd
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_cd

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_cd
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_cd, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_cd, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);


};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      5.010
        (2.401)
        [0.037]
---------------
Eff~c   1274.997
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      2.009
        (0.985)
        [0.041]
---------------
Eff~c   1274.997
$\t~%     23.109
Obs~n       1241
---------------

. 


## Alternative dependent variables

### Inverse hyperbolic sine

In [40]:
%%stata
qui{


foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)
sum D.stay_home
local D_stay_home_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome_ihs="(stay_home: _b[D.stay_home]*((sqrt(`suicide_f_m'^2+1))/`suicide_f_m')) ";
local paracomp_m_stayhome_ihs="(stay_home: _b[D.stay_home]*((sqrt(`suicide_m_m'^2+1))/`suicide_m_m')) ";
    
#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_ihs (D.stay_home=D.commuting_time_iv), gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_ihs

** Save 2nd stage result
nlcom `paracomp_f_stayhome_ihs', post
est store suicide_f_`x'_ct_2nd_ihs

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_ihs

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_ihs

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_ihs
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_ihs

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_ihs

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_ihs (D.stay_home=D.commuting_time_iv), gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_ihs

** Save 2nd stage result
nlcom `paracomp_m_stayhome_ihs', post
est store suicide_m_`x'_ct_2nd_ihs

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_ihs

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_ihs

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_ihs
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_ihs

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_ihs
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_ihs, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_ihs, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);
};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      5.319
        (2.206)
        [0.016]
---------------
Eff~c   1964.183
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      1.377
        (0.712)
        [0.053]
---------------
Eff~c   1964.183
$\t~%     23.109
Obs~n       1241
---------------

. 


## Alternative IV: SOE

In [41]:
%%stata
qui{
foreach x in u20 40_49 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_m_stayhome="(stay_home: _b[D.stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";

#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D.stay_home=D_state_emergency_rate), gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_f_`x'_ct_SOE

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd_SOE

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_`x'_ct_1st_SOE

** Add 1st stage statistics
est restore tmp_suicide_f_`x'_ct_SOE

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_f_`x'_ct_2nd_SOE
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_f_`x'_ct_2nd_SOE

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_f_`x'_ct_2nd_SOE

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D.stay_home=D_state_emergency_rate), gmm2s first cluster(`clsvars') savefprefix(F_1st_)
est store tmp_suicide_m_`x'_ct_SOE

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd_SOE

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_`x'_ct_1st_SOE

** Add 1st stage statistics
est restore tmp_suicide_m_`x'_ct_SOE

weakivtest
estadd scalar F_eff = r(F_eff) : suicide_m_`x'_ct_2nd_SOE
estadd scalar c_TSLS_10 = r(c_TSLS_10) : suicide_m_`x'_ct_2nd_SOE

// ANDERSON-RUBIN
estadd scalar ar_chi=`e(archi2)' : suicide_m_`x'_ct_2nd_SOE
}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd_SOE, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("Under 20") varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_40_49_ct_2nd_SOE, 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
mlabels("40-49") varwidth(4) modelwidth(10);
};


. qui{
******* Female: Commuing-time SSIV *******

---------------
            (1)
       Under 20
         b/se/p
---------------
Sta~x      5.260
        (2.478)
        [0.034]
---------------
Eff~c    715.280
$\t~%     23.109
Obs~n       1241
---------------
******* Male: Commuing-time SSIV *******

---------------
            (1)
          40-49
         b/se/p
---------------
Sta~x      1.081
        (0.905)
        [0.232]
---------------
Eff~c    715.280
$\t~%     23.109
Obs~n       1241
---------------

. 


## Export Robustness

## Female under 20

- column (1): Commuting-Time-Telework-ShiftShare 
- column (2): city Level
- column (3): Industry Level
- column (4): city Level + Industry Level + Other Controls (Age-Group/Employment-Status-Group/Single-Household-Ratio) 
- column (5): Commuting-Time-Telework-ShiftShare + Δunemployment
- column (6): Commuting-Time-Telework-ShiftShare + Δactive job opening ratio
- column (7): Commuting-Time-Telework-ShiftShare + Δunemployment + Δactive job opening ratio
- column (8): Commuting-Time-Telework-ShiftShare + Δcovid cases ＋ Δcovid death cases
- column (9): Commuting-Time-Telework-ShiftShare + Inverse Hyperbolic Sine

In [42]:
%%stata

#delimit ;
noi di "******* Female: Commuting-time SSIV *******";

esttab 
    suicide_f_u20_ct_2nd 
    suicide_f_u20_ct_2nd_city 
    suicide_f_u20_ct_2nd_ind 
    suicide_f_u20_ct_2nd_cio 
    suicide_f_u20_ct_2nd_uemp 
    suicide_f_u20_ct_2nd_ajor 
    suicide_f_u20_ct_2nd_ua
    suicide_f_u20_ct_2nd_cd
    suicide_f_u20_ct_2nd_ihs
    suicide_f_u20_ct_2nd_SOE
, cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
  star(* 0.10 ** 0.05 *** 0.01) 
  label
  stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0fc) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
  varwidth(7) 
  modelwidth(6);


. 
. #delimit ;
delimiter now ;
. noi di "******* Female: Commuting-time SSIV *******";
******* Female: Commuting-time SSIV *******

. esttab 
>     suicide_f_u20_ct_2nd 
>     suicide_f_u20_ct_2nd_city 
>     suicide_f_u20_ct_2nd_ind 
>     suicide_f_u20_ct_2nd_cio 
>     suicide_f_u20_ct_2nd_uemp 
>     suicide_f_u20_ct_2nd_ajor 
>     suicide_f_u20_ct_2nd_ua
>     suicide_f_u20_ct_2nd_cd
>     suicide_f_u20_ct_2nd_ihs
>     suicide_f_u20_ct_2nd_SOE
> , cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
>   star(* 0.10 ** 0.05 *** 0.01) 
>   label
>   stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0fc) labels("Effective F 
> statistic" "$\tau$=10\%" "Observation"))
>   varwidth(7) 
>   modelwidth(6);

-----------------------------------------------------------------------------
           (1)    (2)    (3)    (4)    (5)    (6)    (7)    (8)    (9)   (10)
        D.suic.. D.suic.. D.suic.. D.suic.. D.suic.. D.suic.. D.suic.. D.suic..
>  D.suic.. D.suic..
        b/se/p b/se/p b/s

In [43]:
%%stata
* Export stacked tables
qui{
#delimit ;
/* Female */;
/* commuting-ratio shift-share IV */;
esttab 
suicide_f_u20_ct_2nd suicide_f_u20_ct_2nd_city suicide_f_u20_ct_2nd_ind suicide_f_u20_ct_2nd_cio suicide_f_u20_ct_2nd_uemp suicide_f_u20_ct_2nd_ajor suicide_f_u20_ct_2nd_ua suicide_f_u20_ct_2nd_ihs suicide_f_u20_ct_2nd_SOE
using "../figuretable/robustness/robustness_stayhome_female_u20.tex",
prehead("\begin{tabular}{l*{8}{c}} \toprule") 
fragment
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01)
stats(ar_chi N, layout(@ @) fmt(3 %5.0f) labels("Anderson-Rubin $\chi^2$" "Observation")) 
nomtitles noobs label
booktabs alignment(D{.}{.}{-1})
prefoot("\hline") 
postfoot("\bottomrule \multicolumn{9}{l}{\footnotesize Standard errors in parentheses}\\ \multicolumn{5}{l}{\footnotesize \sym{*} \(p<0.10\), \sym{**} \(p<0.05\), \sym{***} \(p<0.01\)} \end{tabular}") 
replace;
};


. * Export stacked tables
. qui{

. 


## Male 40-49

In [44]:
%%stata

#delimit ;
noi di "******* Male: Commuting-time SSIV *******";

esttab 
    suicide_m_40_49_ct_2nd 
    suicide_m_40_49_ct_2nd_city 
    suicide_m_40_49_ct_2nd_ind 
    suicide_m_40_49_ct_2nd_cio 
    suicide_m_40_49_ct_2nd_uemp 
    suicide_m_40_49_ct_2nd_ajor 
    suicide_m_40_49_ct_2nd_ua
    suicide_m_40_49_ct_2nd_cd
    suicide_m_40_49_ct_2nd_ihs
, cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
  star(* 0.10 ** 0.05 *** 0.01) 
  label
  stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F statistic" "$\tau$=10\%" "Observation"))
  varwidth(7) 
  modelwidth(7);


. 
. #delimit ;
delimiter now ;
. noi di "******* Male: Commuting-time SSIV *******";
******* Male: Commuting-time SSIV *******

. esttab 
>     suicide_m_40_49_ct_2nd 
>     suicide_m_40_49_ct_2nd_city 
>     suicide_m_40_49_ct_2nd_ind 
>     suicide_m_40_49_ct_2nd_cio 
>     suicide_m_40_49_ct_2nd_uemp 
>     suicide_m_40_49_ct_2nd_ajor 
>     suicide_m_40_49_ct_2nd_ua
>     suicide_m_40_49_ct_2nd_cd
>     suicide_m_40_49_ct_2nd_ihs
> , cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
>   star(* 0.10 ** 0.05 *** 0.01) 
>   label
>   stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) labels("Effective F s
> tatistic" "$\tau$=10\%" "Observation"))
>   varwidth(7) 
>   modelwidth(7);

-------------------------------------------------------------------------------
            (1)     (2)     (3)     (4)     (5)     (6)     (7)     (8)     (9)
        D.sui.. D.sui.. D.sui.. D.sui.. D.sui.. D.sui.. D.sui.. D.sui.. D.sui..
         b/se/p  b/se/p  b/se/p  b/se/p  b/se/p  b/se/p  b/s

In [45]:
%%stata
* Export stacked tables
qui{
#delimit ;
/* Female */;
/* commuting-ratio shift-share IV */;
esttab 
suicide_m_40_49_ct_2nd suicide_m_40_49_ct_2nd_city suicide_m_40_49_ct_2nd_ind suicide_m_40_49_ct_2nd_cio suicide_m_40_49_ct_2nd_uemp suicide_m_40_49_ct_2nd_ajor suicide_m_40_49_ct_2nd_ua suicide_m_40_49_ct_2nd_ihs suicide_m_40_49_ct_2nd_SOE
using "../figuretable/robustness/robustness_stayhome_male_40_49.tex",
prehead("\begin{tabular}{l*{8}{c}} \toprule") 
fragment
b(%4.3f) se(%4.3f) star(* 0.10 ** 0.05  *** 0.01)
stats(ar_chi N, layout(@ @) fmt(3 %5.0f) labels("Anderson-Rubin $\chi^2$" "Observation")) 
nomtitles noobs label
booktabs alignment(D{.}{.}{-1})
prefoot("\hline") 
postfoot("\bottomrule \multicolumn{9}{l}{\footnotesize Standard errors in parentheses}\\ \multicolumn{5}{l}{\footnotesize \sym{*} \(p<0.10\), \sym{**} \(p<0.05\), \sym{***} \(p<0.01\)} \end{tabular}") 
replace;
};


. * Export stacked tables
. qui{

. 
